# 올리브영 상세 정보 및 리뷰 크롤링

In [1]:
import pandas as pd
# # 모든 열 너비를 표시
# pd.set_option('display.max_colwidth', None)  
# # 옵션을 기본값으로 재설정
# pd.reset_option('display.max_colwidth')

In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

CHROME_DRIVER_PATH = "C:\\Users\\82108\\.wdm\\drivers\\chromedriver\\win64\\127.0.6533.72\\chromedriver-win32\\chromedriver.exe"
service = Service(executable_path=CHROME_DRIVER_PATH)
options = Options()
driver = webdriver.Chrome(service=service, options=options)

name_list = []
brand_list = []
price_list = [] 
sale_price_list = []
picture_list = [] 
url_list = []
volume_list = []
skin_type_list = []
ingredient_list = []
review_list = []
all_review_list = []

# 각 제품을 빈 리스트에 추가
def make_list(name, brand, price, sale_price, picture, url, volume, skin_type, ingredient, indiv_product_reviews):
    name_list.append(name)
    brand_list.append(brand)
    price_list.append(price)
    sale_price_list.append(sale_price)
    picture_list.append(picture)
    url_list.append(url)
    volume_list.append(volume)
    skin_type_list.append(skin_type)
    ingredient_list.append(ingredient)
    review_list.append(indiv_product_reviews)

# 리뷰 페이지 내에서 다음 페이지 클릭
def review_next_page(review_page_num):
    try:
        page_bar = driver.find_elements(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing a')
        for button in page_bar:
            if button.get_attribute("data-page-no") == str(review_page_num):
                driver.execute_script("arguments[0].click();", button)
                # WebDriverWait(driver, 10).until(
                #     EC.presence_of_element_located((By.CSS_SELECTOR, '#gdasList')))
                
                break
    except NoSuchElementException:
        pass
    

# 리뷰 추출하는 코드
def extract_review(driver,name):
    # 최신순으로 정렬하는 코드
    try: 
        sort_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="gdasSort"]/li[3]/a')))
        driver.execute_script("arguments[0].click();", sort_button)
        WebDriverWait(driver, 5)
    
        # 체험단 제거하는 코드
        ad_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="searchType_3"]')))
        ad_button.click()    
    
        # 모든 페이지 버튼 요소를 찾기
        page_buttons = driver.find_elements(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing a')
        # 각 제품의 리뷰 하나의 리스트로 담기
        indiv_product_reviews = []
        # 리뷰 페이지에 한 페이지 당 10개의 리뷰 존재
        for review_page_num in range(1, 11): 
            try:
                review_next_page(review_page_num)
                WebDriverWait(driver, 60).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#gdasList > li > div.review_cont > div.txt_inner')))
                for review_num in range(1, 11):
                    # 몇개의 리뷰에서 (리뷰 + 타이틀, 리뷰, 타이틀) >> 3개의 형식으로 리뷰가 작성됨
                    # 리뷰와 타이틀을 더해줌
                    # 특정 날짜 이후로 타이틀이 없어지고, 그 타이틀 패스가 바로 및의 문장으로 변경 됨
                    #  ※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후~~
                    # 전처리할 때 없애줘야 할거 같음
                    try:
                        # 리뷰 타이틀과 내용을 초기화
                        review_title = ""
                        review_content = ""

                        # 리뷰 타이틀 추출
                        try:
                            review_title = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({review_num}) > div.review_cont > p').text
                        except (NoSuchElementException, StaleElementReferenceException):
                            pass

                        # 리뷰 내용 추출
                        try:
                            review_content = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({review_num}) > div.review_cont > div.txt_inner').text
                        except (NoSuchElementException, StaleElementReferenceException):
                            pass

                        # 타이틀과 내용을 합쳐서 리뷰 리스트에 추가
                        if review_title or review_content:
                            review = [f"{review_title} {review_content}".strip()]
                            indiv_product_reviews.append(review)
                            all_review_list.append(review)

                    except (NoSuchElementException, StaleElementReferenceException):
                        pass
            except (NoSuchElementException, StaleElementReferenceException):
                break
#        indiv_product_reviews = set(indiv_product_reviews)     
        return indiv_product_reviews
    except:
        print(name)
        

# 제품 내 모든 정보 추출
# para >> 제품 목록에 들어가면 문단(행?)으로 path가 달라짐
# 첫 페이지에서 48개의 제품이 보이게 했을 때 문단 8~19의 숫자로 path가 달라짐
# product_number >> 열 방향?으로도 달라짐.가로 방향으로 1~4씩 바뀜  
def extract_data(driver, para, product_number):
    try:
        # 제품 클릭
        product_button = driver.find_element(By.CSS_SELECTOR, f'#Contents > ul:nth-child({para}) > li:nth-child({product_number}) > div > div > a > p') 
        product_button.click()
        
        # 제품 명 추출
        name = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name'))).text
        
        # url 추출
        url = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'head > link')))
        url = url.get_attribute('href')
        
        # 이미지 추출
        # 2번째 이미지 먼저 추출. 없을 시 첫 번째 이미지 추출
        try:
            # img 요소를 찾기(두 번째 사진)
            img_element = driver.find_element(By.CSS_SELECTOR, '#prd_thumb_list > li:nth-child(3) > a')
            # src 속성 값 가져오기
            picture = img_element.get_attribute('data-img')
        except:
            # img 요소를 찾기
            img_element = driver.find_element(By.ID, 'mainImg')
            # src 속성 값 가져오기
            picture = img_element.get_attribute('src')

        # 브랜드 명 추출
        brand = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#moveBrandShop'))).text

        # 가격 추출
        # 할인된 가격이 없을 시, None으로 대체
        try:
            sale_price_element = driver.find_element(By.CSS_SELECTOR, '#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-2 > strong')  
            sale_price = sale_price_element.text
            price_element = driver.find_element(By.CSS_SELECTOR, '#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-1 > strike') 
            price = price_element.text
        except NoSuchElementException:
            sale_price = None
            price_element = driver.find_element(By.CSS_SELECTOR, '#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span > strong') 
            price = price_element.text


        # 구매정보 클릭(상세 정보보)
        info_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#buyInfo > a')))
        info_button.click()
        
        # 용량 추출
        volume = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="artcInfo"]/dl[2]/dd'))).text
        
        # 제품주요사양(스킨타입) 추출
        skin_type = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="artcInfo"]/dl[3]/dd'))).text
        
        # 성분 추출
        ingredient = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="artcInfo"]/dl[8]/dd'))).text
        
        # 리뷰 버튼 클릭
        review_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#reviewInfo > a')))
        review_button.click()
        
        # 각 제품의 정보 출력
        indiv_product_reviews = extract_review(driver, name)
        
        # 한 제품에 대한 리스트 만들기
        make_list(name, brand, price, sale_price, picture, url, volume, skin_type, ingredient, indiv_product_reviews)
        driver.back()
        
    except NoSuchElementException:
        driver.back()


try:
    # 크롤링을 중지할 조건
    # True로 변경되면 중지
    stop = False
    # 페이지 1~8
    for page in range(1, 9):
        if stop:
            break
        for para in range(8,20):
            if stop:
                break
            for product_number in range(1, 5):
                if stop:
                    break
                url = f'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010013&fltDispCatNo=&prdSort=01&pageIdx={page}&rowsPerPage=48&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=&bShowCnt=&cShowCnt=&trackingCd=Cat100000100010013_Small&amplitudePageGubun=&t_page=&t_click=&midCategory=%EC%8A%A4%ED%82%A8%2F%ED%86%A0%EB%84%88&smallCategory=%EC%A0%84%EC%B2%B4&checkBrnds=&lastChkBrnd='
                driver.get(url)

                try:
                    extract_data(driver, para, product_number)
                except NoSuchElementException:
                    stop = True
                    break
    
    # 데이터프레임 생성 및 출력
    info_data = {'name': name_list,'brand': brand_list, 'price': price_list, 'sale_price': sale_price_list, 'picture':picture_list, 'url' : url_list, 'volume': volume_list, 'skin_type': skin_type_list, 'ingredient': ingredient_list, 'review' : review_list}
    df_info = pd.DataFrame(info_data)
    # 중복 있는지 확인용
    all_review = {'review':all_review_list}
    df_all_review = pd.DataFrame(all_review)
finally:
    # 브라우저 종료
    driver.quit()     



제나벨 PDRN 리쥬비네이팅 토너 300ml
비욘드 엔젤아쿠아 수분진정 토너 500ml 기획 (+화장솜 70매)
스콧해미쉬 블루 블렌디드 토너 에센스 180ml
크리니크 안티-블레미쉬 솔루션 클래리파잉 로션 200ml
더바디샵 티트리 스킨 클리어링 토너 250ML
오딧세이 챕터파이브 안티옥시던트 생츄어리 토너 140ml
컴포트존 리바이탈라이징 토너 200ml
메디톡스 뉴라덤 뉴로락토 토너 150ml
더바디샵 티트리 래피드 액션 젤 15ML
[영국프리미엄 스킨케어] 닐스야드 레머디스 와일드 로즈 아하 토너 200ml
더후 공진향 탄력 밸런싱 토너 150ml
더후 로얄레지나 에너지드롭 트리트먼트75ml
[영국프리미엄 스킨케어] 닐스야드 레머디스 와일드 로즈 아하 토너 200ml
메디톡스 뉴라덤 뉴로락토 토너 150ml
더바디샵 티트리 래피드 액션 젤 15ML
더후 로얄레지나 에너지드롭 트리트먼트75ml
더랩바이블랑두 저분자 히알루론산 딥 토너 리필캔 238ml
더후 공진향 탄력 밸런싱 토너 150ml
비욘드 엔젤아쿠아 수분진정 토너 500ml 기획 (+화장솜 70매)
셀린저 리얼 토너 카렌듈라 200ml


In [32]:
df_info

,name,brand,price,sale_price,picture,url,volume,skin_type,ingredient,review
0,[단독기획] 토리든 다이브인 저분자 히알루론산 토너 500ml 기획 (+화장솜 60...,토리든,"29,000","19,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,[본품] 다이브인 토너 500ml [증정] 화장솜 60매,모든 피부 타입,"1. 다이브인 저분자 히알루론산 토너 500ml (본품)\n정제수, 부틸렌글라이콜,...","[[닦토로 써도되고 용량도 커서 사용하게 무난한 수분토너인것갘아여], [토리든 수분..."
1,[쿨링진정]넘버즈인 1번 진정 맑게담은 청초토너 300ml 리필기획(+300ml 증정),넘버즈인,"40,000","26,600",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,[본품] 300ml / [리필] 300ml,모든 피부 용,"정제수, 1,2-헥산다이올, 다이프로필렌글라이콜, 프로판다이올, 부틸렌글라이콜, 글...","[[순하고 자극이 아예 없어사 사용하기 너무 좋아용! 여름에도 좋네요], [세일 안..."
2,[1등토너] 라운드랩 1025 독도 토너 500ml 기획 (+200ml+ 마스크팩 1매),라운드랩,"45,000","29,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,[본품] 500ml+200ml + [증정품] 마스크팩 1매,모든 피부용,"[독도 토너]\n정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, ...","[[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별..."
3,[7월 올영픽/쿨링진정] 아비브 어성초 카밍 토너 스킨부스터 더블 기획 (200ml...,아비브,"39,000","25,940",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,200ml+200ml,모든 피부용,"정제수, 약모밀추출물, 글리세린, 부틸렌글라이콜, 펜틸렌글라이콜, 자일리톨, 비피다...","[[추천 받아서 사본 제품인데 괜찮은 거 같아요. 꾸준히 사용해 볼 예정입니다], ..."
4,아벤느 젠틀 토닝 로션 200ml 더블기획,아벤느,"40,500","34,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,본품 200ml + 본품 200ml,모든 피부 타입 / 수부지 피부,아벤느 온천수 (아쿠아)\n향료\n하이드로제네이티드스타치하이드롤리세이트\n피이지-7...,"[[엄청 좋은건 모르겠고 순해서 솜에 적셔서 얼굴에 올려두기 좋음], [아벤느 제품..."
...,...,...,...,...,...,...,...,...,...,...
351,디오디너리 랙틱 애시드 5%+에이치에이,디오디너리,"11,800",None,https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,30ml,모든피부타입,"정제수, 락틱애씨드, 글리세린, 펜틸렌글라이콜, 프로판다이올, 소듐하이알루로네이트크...",[[순한 락트산(AHA) 각질제거 포뮬라 락트산은 알파 하이드록시산(AHA)으로 피...
352,라로슈포제 똘러리앙 울트라 로션 400ML,라로슈포제,"48,000","40,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,200ml,아시아피부 사용적합 / 민감피부 사용적합,정제수 글리세린 다이프로필렌글라이콜 부틸렌글라이콜 프로필렌글라이콜 펜틸렌글라이콜 나...,"[[가격대는 좀 있지만 촉촉해져서 진짜 너무 좋아요\n재구매 의사 있음], [순하고..."
353,[단독기획] 스킨푸드 캐롯 카로틴 수딩 토너 300ml+200ml,스킨푸드,"30,000",None,https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,300ml+200ml,모든피부,"정제수, 당근추출물, 글리세린, 부틸렌글라이콜, 1,2-헥산다이올, 피피지-13-데...",[[속건조까지 확실히 잡아주는 느낌의 엄청 촉촉한 물토너까지는 아닌데 스킨푸드 당근...
354,아벤느 이드랑스 딥모이스트 로션 200ml 더블 기획,아벤느,"40,500","35,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,아벤느 이드랑스 딥 모이스트 로션 200ml 본품 2개 / 총 400ml,모든 피부 타입 / 건성 피부 / 수분 부족형 지성 피부,"아쿠아(아벤느 온천수)\n글리세린\n메틸글루세스-20\n펜틸렌글라이콜\n1,2-헥산...","[[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별..."


In [33]:
df_info.to_csv('skin_toner_review.csv')

In [48]:
df_info.drop_duplicates(subset='name')

,name,brand,price,sale_price,picture,url,volume,skin_type,ingredient,review
0,[단독기획] 토리든 다이브인 저분자 히알루론산 토너 500ml 기획 (+화장솜 60...,토리든,"29,000","19,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,[본품] 다이브인 토너 500ml [증정] 화장솜 60매,모든 피부 타입,"1. 다이브인 저분자 히알루론산 토너 500ml (본품)\n정제수, 부틸렌글라이콜,...","[[닦토로 써도되고 용량도 커서 사용하게 무난한 수분토너인것갘아여], [토리든 수분..."
1,[쿨링진정]넘버즈인 1번 진정 맑게담은 청초토너 300ml 리필기획(+300ml 증정),넘버즈인,"40,000","26,600",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,[본품] 300ml / [리필] 300ml,모든 피부 용,"정제수, 1,2-헥산다이올, 다이프로필렌글라이콜, 프로판다이올, 부틸렌글라이콜, 글...","[[순하고 자극이 아예 없어사 사용하기 너무 좋아용! 여름에도 좋네요], [세일 안..."
2,[1등토너] 라운드랩 1025 독도 토너 500ml 기획 (+200ml+ 마스크팩 1매),라운드랩,"45,000","29,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,[본품] 500ml+200ml + [증정품] 마스크팩 1매,모든 피부용,"[독도 토너]\n정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, ...","[[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별..."
3,[7월 올영픽/쿨링진정] 아비브 어성초 카밍 토너 스킨부스터 더블 기획 (200ml...,아비브,"39,000","25,940",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,200ml+200ml,모든 피부용,"정제수, 약모밀추출물, 글리세린, 부틸렌글라이콜, 펜틸렌글라이콜, 자일리톨, 비피다...","[[추천 받아서 사본 제품인데 괜찮은 거 같아요. 꾸준히 사용해 볼 예정입니다], ..."
4,아벤느 젠틀 토닝 로션 200ml 더블기획,아벤느,"40,500","34,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,본품 200ml + 본품 200ml,모든 피부 타입 / 수부지 피부,아벤느 온천수 (아쿠아)\n향료\n하이드로제네이티드스타치하이드롤리세이트\n피이지-7...,"[[엄청 좋은건 모르겠고 순해서 솜에 적셔서 얼굴에 올려두기 좋음], [아벤느 제품..."
...,...,...,...,...,...,...,...,...,...,...
346,빌리프 슈퍼 드랍스 비타민C 워터트리트먼트 150ML,빌리프,"42,000","31,920",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,150ml,모든피부용,"정제수, 프로판다이올, 글리세린* (4%), 변성알코올, 1,2-헥산다이올, 다이페...","[[할인하길래 좋대서 구매해봅니다 관련 상품들 함께 구매했어요], [선물주려고 산건..."
347,플리프 시카-알로에 프레쉬 스킨 200ml,플리프,"18,000",None,https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,■ 200ml,모든 피부타입,"알로에베라잎추출물,부틸렌글라이콜,병풀추출물,나이아신아마이드,베타인,프로판다이올,안젤...","[[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별..."
348,비오템옴므 아쿠아틱 로션 애프터 쉐이브 200ml,비오템 옴므,"61,000","51,850",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,200ml,모든 피부,"정제수, 변성알코올, 글리세린, 부틸렌글라이콜, 향료, 아크릴레이트/C10-30알킬...",[[스킨로션 찾던중 올영에 비오템이 있길래 망설임없이 주문했습니다. 근데 받고나니 ...
349,[한정기획]라운드랩 1025 독도 토너 대용량 리필기획 (300ml+300ml 리필...,라운드랩,"33,000","29,600",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,[본품]1025 독도 토너 300ml+[구성품]1025 독도 토너 300ml 리필+...,모든 피부용,"[라운드랩 1025 독도 토너]\n정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜...","[[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별..."


In [34]:
df_all_review[:]

,review
0,[닦토로 써도되고 용량도 커서 사용하게 무난한 수분토너인것갘아여]
1,[토리든 수분라인은 믿고 사용합니다! 저렴하고 양도 많아서 닥토로 제격이에요!]
2,[눈시림 없고 촉촉해서 잘 쓰고 있어요 좋아요 꾸준히 구매할 거 같어요 추천합니다]
3,"[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ..."
4,[토너팩이랑 닥토로 사용했었는데 되게 순하고 좋았어요!!향은 무향..? 호불호 갈릴...
...,...
30011,"[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ..."
30012,"[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ..."
30013,"[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ..."
30014,"[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ..."


In [35]:
df_all_review.review.drop_duplicates()

0                     [닦토로 써도되고 용량도 커서 사용하게 무난한 수분토너인것갘아여]
1             [토리든 수분라인은 믿고 사용합니다! 저렴하고 양도 많아서 닥토로 제격이에요!]
2           [눈시림 없고 촉촉해서 잘 쓰고 있어요 좋아요 꾸준히 구매할 거 같어요 추천합니다]
3        [※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ...
4        [토너팩이랑 닥토로 사용했었는데 되게 순하고 좋았어요!!향은 무향..? 호불호 갈릴...
                               ...                        
29573    [※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ...
29701    [라포슈포제밑고 사용합니다 셈플써보고 제구입했구요 가격할인해서 잘샀어요 촉촉하고 모...
29827    [※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ...
29857    [※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ...
29957    [※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ...
Name: review, Length: 21530, dtype: object

In [49]:
30016 - 29958

58

In [52]:
# 올리브영 내에서 정렬이 계속 변하면서, 중복되는 제품 발생
df_info.drop_duplicates(subset='name')

,name,brand,price,sale_price,picture,url,volume,skin_type,ingredient,review
0,[단독기획] 토리든 다이브인 저분자 히알루론산 토너 500ml 기획 (+화장솜 60...,토리든,"29,000","19,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,[본품] 다이브인 토너 500ml [증정] 화장솜 60매,모든 피부 타입,"1. 다이브인 저분자 히알루론산 토너 500ml (본품)\n정제수, 부틸렌글라이콜,...","[[닦토로 써도되고 용량도 커서 사용하게 무난한 수분토너인것갘아여], [토리든 수분..."
1,[쿨링진정]넘버즈인 1번 진정 맑게담은 청초토너 300ml 리필기획(+300ml 증정),넘버즈인,"40,000","26,600",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,[본품] 300ml / [리필] 300ml,모든 피부 용,"정제수, 1,2-헥산다이올, 다이프로필렌글라이콜, 프로판다이올, 부틸렌글라이콜, 글...","[[순하고 자극이 아예 없어사 사용하기 너무 좋아용! 여름에도 좋네요], [세일 안..."
2,[1등토너] 라운드랩 1025 독도 토너 500ml 기획 (+200ml+ 마스크팩 1매),라운드랩,"45,000","29,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,[본품] 500ml+200ml + [증정품] 마스크팩 1매,모든 피부용,"[독도 토너]\n정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, ...","[[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별..."
3,[7월 올영픽/쿨링진정] 아비브 어성초 카밍 토너 스킨부스터 더블 기획 (200ml...,아비브,"39,000","25,940",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,200ml+200ml,모든 피부용,"정제수, 약모밀추출물, 글리세린, 부틸렌글라이콜, 펜틸렌글라이콜, 자일리톨, 비피다...","[[추천 받아서 사본 제품인데 괜찮은 거 같아요. 꾸준히 사용해 볼 예정입니다], ..."
4,아벤느 젠틀 토닝 로션 200ml 더블기획,아벤느,"40,500","34,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,본품 200ml + 본품 200ml,모든 피부 타입 / 수부지 피부,아벤느 온천수 (아쿠아)\n향료\n하이드로제네이티드스타치하이드롤리세이트\n피이지-7...,"[[엄청 좋은건 모르겠고 순해서 솜에 적셔서 얼굴에 올려두기 좋음], [아벤느 제품..."
...,...,...,...,...,...,...,...,...,...,...
346,빌리프 슈퍼 드랍스 비타민C 워터트리트먼트 150ML,빌리프,"42,000","31,920",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,150ml,모든피부용,"정제수, 프로판다이올, 글리세린* (4%), 변성알코올, 1,2-헥산다이올, 다이페...","[[할인하길래 좋대서 구매해봅니다 관련 상품들 함께 구매했어요], [선물주려고 산건..."
347,플리프 시카-알로에 프레쉬 스킨 200ml,플리프,"18,000",None,https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,■ 200ml,모든 피부타입,"알로에베라잎추출물,부틸렌글라이콜,병풀추출물,나이아신아마이드,베타인,프로판다이올,안젤...","[[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별..."
348,비오템옴므 아쿠아틱 로션 애프터 쉐이브 200ml,비오템 옴므,"61,000","51,850",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,200ml,모든 피부,"정제수, 변성알코올, 글리세린, 부틸렌글라이콜, 향료, 아크릴레이트/C10-30알킬...",[[스킨로션 찾던중 올영에 비오템이 있길래 망설임없이 주문했습니다. 근데 받고나니 ...
349,[한정기획]라운드랩 1025 독도 토너 대용량 리필기획 (300ml+300ml 리필...,라운드랩,"33,000","29,600",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,[본품]1025 독도 토너 300ml+[구성품]1025 독도 토너 300ml 리필+...,모든 피부용,"[라운드랩 1025 독도 토너]\n정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜...","[[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별..."
